In [34]:
import re
from dataclasses import dataclass
from copy import deepcopy, copy
from functools import reduce

class part:
    @dataclass
    class mm:
        max: int = 4001
        min: int = 0
        
        def __repr__(self) -> str:
            return f"({self.min},{self.max})"
    
        def span(self):
            return self.max - self.min
    
    def __init__(self) -> None:
        self.x = part.mm()
        self.m = part.mm()
        self.a = part.mm()
        self.s = part.mm()
        
    def __repr__(self) -> str:
        return f"{{x={self.x},m={self.m},a={self.a},s={self.s}}}"
    
    def __getitem__(self, key: str) -> mm:
        match key:
            case 'x':
                return self.x
            case 'm':
                return self.m
            case 'a':
                return self.a
            case 's':
                return self.s
            case _:
                raise KeyError(f"part has no attribute '{key}'")
            
    def value(self):
        return self.x.span() * self.m.span() * self.a.span() * self.s.span()
    

with open("input-small.txt") as f:
    lines = f.readlines()
    
empty_index = lines.index("\n")
workflows = {}
for line in lines[:empty_index]:
    m = re.match(r"^([a-z]+)\{(.+)\}$", line)
    identifier = m.group(1)
    workflows[identifier] = []
    rules = m.group(2).split(",")
    for rule in rules:
        m = re.match(r"(a|s|m|x)(<|>)(\d+):([a-z]+|A|R)", rule)
        if not m:
            workflows[identifier].append(rule)
        else:
            workflows[identifier].append([
                m.group(1),
                m.group(2),
                int(m.group(3)),
                m.group(4)
            ])
            
predecessors = {workflow: [] for workflow in workflows}
predecessors['A'] = []
for workflow in workflows:
    for index, rule in enumerate(workflows[workflow]):
        if(type(rule) == str):
            if rule != 'R':
                predecessors[rule].append((workflow, index))
        else:
            if rule[-1] != 'R':
                predecessors[rule[-1]].append((workflow, index+1))

print(predecessors)
final_parts = []
current_parts = [(part(), predecessors['A'])]
while current_parts:
    print(current_parts)
    new_current_parts = []
    for current_part, rules in current_parts:
        for identifier, index in rules:
            current_part = deepcopy(current_part)
            for [attr, comp, val, next] in workflows[identifier][:index][::-1]:
                if comp == '<':
                    current_part[attr].min = max(val, current_part[attr].min)
                elif comp == '>':
                    current_part[attr].max = min(val, current_part[attr].max)
            
            if identifier == 'in':
                final_parts.append(current_part)
            else:
                new_current_parts.append((current_part, predecessors[identifier]))
    current_parts = new_current_parts

print(final_parts)
reduce(lambda a, b: a + b.value(), final_parts, 0)
    

{'px': [('in', 1)], 'pv': [('hdj', 1)], 'lnx': [('qs', 1)], 'rfg': [('px', 2)], 'qs': [('qqz', 1)], 'qkq': [('px', 1)], 'crn': [('qkq', 1)], 'in': [], 'qqz': [('in', 1)], 'gd': [('rfg', 1)], 'hdj': [('qqz', 2)], 'A': [('px', 2), ('pv', 1), ('lnx', 1), ('lnx', 1), ('rfg', 2), ('qs', 1), ('qkq', 1), ('crn', 1), ('hdj', 1)]}
[({x=(0,4001),m=(0,4001),a=(0,4001),s=(0,4001)}, [('px', 2), ('pv', 1), ('lnx', 1), ('lnx', 1), ('rfg', 2), ('qs', 1), ('qkq', 1), ('crn', 1), ('hdj', 1)])]
[({x=(0,4001),m=(0,2090),a=(2006,4001),s=(0,4001)}, [('in', 1)]), ({x=(0,4001),m=(0,2090),a=(2006,1716),s=(0,4001)}, [('hdj', 1)]), ({x=(0,4001),m=(0,1548),a=(2006,1716),s=(0,4001)}, [('qs', 1)]), ({x=(0,4001),m=(0,1548),a=(2006,1716),s=(0,4001)}, [('qs', 1)]), ({x=(0,2440),m=(0,1548),a=(2006,1716),s=(537,4001)}, [('px', 2)]), ({x=(0,2440),m=(0,1548),a=(2006,1716),s=(537,3448)}, [('qqz', 1)]), ({x=(1416,2440),m=(0,1548),a=(2006,1716),s=(537,3448)}, [('px', 1)]), ({x=(1416,2440),m=(0,1548),a=(2006,1716),s=(537,3448

34717196843070